In [1]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "-1"
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import tensorflow_hub as hub
import numpy
import spacy
import re
from utils.ngram import ngram
from utils.create_hash import create_word_to_int, create_int_to_word

In [2]:
if (1):
    rf = open("./data/hp0.txt", "r", encoding="utf8")
    wf = open("./data/hp1.txt", "w", encoding="utf8")
    for line in rf.readlines():
        if (re.match("Page | . Harry Potter and the Philosophers Stone - J.K. Rowling", line) == None and line != "\n"):
            wf.write(line.replace("\n", " "))
    rf.close()
    wf.close()

In [3]:
clean_punc = open("./data/hp2.txt", "r", encoding="utf8")
punctuation = [".", "\"", "\'", ":", ";", ",", "”", "“", "-", "—", "!", "?", "/", "\\", "(", ")"]
wf = open("./data/hp1.txt", "w", encoding="utf8")
lines = clean_punc.read()
for each in punctuation:
    lines = lines.replace(each, " ")
lines = re.sub(" +", " ", lines)
wf.write(lines)
clean_punc.close()
wf.close()

In [ ]:
def remove_puncs(doc): # doc output is list
    output = []
    for token in doc:
        if (not token.is_punct or token.text != " "):
            output.append(token)
    return output

nlp = spacy.load("en_core_web_sm")
rf = open("./data/hp1.txt", "r", encoding="utf8")
doc = nlp(rf.read())
doc = remove_puncs(doc)

In [ ]:
word_to_int = create_word_to_int(doc)
int_to_word = create_int_to_word(word_to_int)
print("word_to_int and int_to_word create done")

In [ ]:
print(len(doc))
print(word_to_int["THE"])
print(int_to_word[6692])

In [ ]:
word_ngram = ngram(doc, 7, word_to_int)
print("create n gram done")

In [ ]:
embed = hub.load("https://tfhub.dev/google/universal-sentence-encoder/4")

In [ ]:
print(word_ngram[0:10])

In [ ]:
embedded_ngram = embed(list(zip(*word_ngram))[0])
labels = list(zip(*word_ngram))[1]

In [ ]:
print(len(word_ngram))
print(embedded_ngram.shape)

In [ ]:
inputs = keras.Input(shape=(512,))
num_classes = len(word_to_int)
x = layers.Dense(3000, activation="relu")(inputs)
outputs = layers.Dense(num_classes, activation="softmax")(x)

model = keras.Model(inputs=inputs, outputs=outputs)
model.summary()

In [ ]:
model.compile(optimizer=keras.optimizers.Adam(learning_rate=3e-4),
              loss=keras.losses.SparseCategoricalCrossentropy())


In [ ]:
model.fit(embedded_ngram.numpy(), numpy.array(labels),
          batch_size=32, epochs=10)

In [53]:
embedded = embed(['Harry, What Where did the Ron'])
outputboi = model.predict(embedded.numpy()) # 1 * 6000
print(outputboi.shape)
outputboi = tf.math.argmax(outputboi, axis=1)
print(int_to_word_loaded[int(outputboi.numpy()[0])])

(1, 6692)
had


In [46]:
model = keras.models.load_model('xd_loss0.4299/xd_loss0.4299')

In [44]:
word_to_int_loaded = pickle.load(open("xd_loss0.4299/xd_loss0.4299/word2int.p", "rb"))
int_to_word_loaded = create_int_to_word(word_to_int_loaded)